In [1]:
import h5py
import sys
import os
import math
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
sys.path.append('/home/ztang/multitask_RNA/data_generation')
import utils 
import numpy as np
celltype = 'K562'

In [2]:
file = h5py.File("/home/ztang/multitask_RNA/data/CAGI/"+celltype+"/onehot.h5", "r")
alt = file['alt']
ref = file['ref']

In [3]:
import nucleotide_transformer
import haiku as hk
import jax
import jax.numpy as jnp
from nucleotide_transformer.pretrained import get_pretrained_model
from tqdm import tqdm
model_name = '2B5_1000G'

if '2B5' in model_name:
    print('2B5_model')
    embed_layer = 32
else:
    print('500M model')
    embed_layer = 24

2B5_model


In [4]:
max_len = math.ceil(len(alt[0])/6)+2
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    mixed_precision=False,
    embeddings_layers_to_save=(embed_layer,),
    attention_maps_to_save=(),
    max_positions=max_len,
)
forward_fn = hk.transform(forward_fn)

In [5]:
alt.shape

(1409, 230, 4)

In [7]:
random_key = jax.random.PRNGKey(0)
N, L, A = alt.shape
batch_size = 200
alt_out = []
ref_out = []
for i in tqdm(range(0,N,batch_size)):
    b_size = batch_size
    if i + batch_size > N:
        b_size = N-i
    onehot = np.concatenate((ref[i:i+b_size],alt[i:i+b_size]))
    seq = utils.onehot_to_seq(onehot)
    token_out = tokenizer.batch_tokenize(seq)
    token_id = [b[1] for b in token_out]
    seq_pair = jnp.asarray(token_id,dtype=jnp.int32)
    outs = forward_fn.apply(parameters, random_key, seq_pair)
    alt_out.extend(outs['embeddings_'+str(embed_layer)][0:b_size])
    ref_out.extend(outs['embeddings_'+str(embed_layer)][b_size:2*b_size])

100%|██████████| 8/8 [00:20<00:00,  2.52s/it]


In [6]:
len(ref_out)

NameError: name 'ref_out' is not defined

In [15]:
output = h5py.File('/home/ztang/multitask_RNA/data/CAGI_embed/'+celltype+'/'+model_name+'.h5', 'w')
output.create_dataset('alt', data=np.array(alt_out))
output.create_dataset('ref', data=np.array(ref_out))
output.close()